# Practical 3: Dgital Elevation Models

In this practical, we'll make visualizations and calculate derived quantities of LiDAR DEMs of the Kīlauea volcano on the Big Island of Hawai'i. The DEMs cover the main volcano summit and the Halemau'mau crater from before and after the [2018 lower East Rift zone eruption](https://www.usgs.gov/volcanoes/kilauea/2018-lower-east-rift-zone-eruption-and-summit-collapse).

[![Aerial photo of Halemau'mau crater and part of the Kīlauea caldera taken on July 13, 2018](https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/img6670.jpg)](https://www.usgs.gov/media/images/aerial-view-k-lauea-summit-july-13-2018)
*Aerial photo of Halemau'mau crater and part of the Kīlauea caldera taken on July 13, 2018. The buildings housed the USGS Hawaiian Volcano Observatory and Hawai‘i Volcanoes National Park's Jaggar Museum. Source: [USGS](https://www.usgs.gov/media/images/aerial-view-k-lauea-summit-july-13-2018) (public domain).*

## Import the required libraries

We'll start by importing the things we'll need to load the images, do some calculations, and plot them.

In [ ]:
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource

## Download the DEMs

The DEMs are available as TIFF files (click to download):

* [`kilauea2009.tif`](https://github.com/leouieda/remote-sensing/raw/main/data/kilauea/kilauea2009.tif) (source: [Hawaii Big Island Survey](https://doi.org/10.5069/G9DZ067X))
* [`kilauea2018.tif`](https://github.com/leouieda/remote-sensing/raw/main/data/kilauea/kilauea2018.tif) (source: [Kilauea Volcano, HI July 2018 Acquisition](https://doi.org/10.5069/G9M32SV1))

Both were generated with **2 meter** grid node spacing and covering the same are (they share the Southwester corner coordinates). The value -9999 indicates missing data and need to be converted to Not-a-Number (NaN) values (see [`numpy.nan`](https://numpy.org/doc/stable/reference/constants.html?highlight=nan#numpy.nan)).

**Place both files in the same folder as this notebook.** After that, we can load them with scikit-image.

### Your turn

Make pseudo-color maps of both DEMs (including a colorbar). Choose an [appropriate colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for these plots. It helps if you ensure that the maximum and minimum values of the colorbar are the same for both plots. You can do this by setting the `vmin` and `vmax` parameters of `plt.imshow`.

## Profiles

It can be helpful for visualization to extract profiles across features of the DEMs. This reduces the dimensions of the DEMs and allows for easier comparison of different datasets (both DEMs can be visualized in the same plot).

For simplicity, we'll extract East-West profiles across the Halemau'mau crater to visualize the summit collapse. We can do this by slicing and indexing the numpy arrays. Extracting profiles along arbitrary lines is a bit more complicated and involves interpolation of the data. In Python, it can be done with the [Verde](https://www.fatiando.org/verde/latest/) library.

### Your turn

Extract the same profile across the 2018 DEM and plot both together in the same figure for comparison.

## Spatial and volumetric analysis

Having co-registered DEMs from before and after the summit collapse allows us to calculate the difference between the two topographic surfaces. 

### Your turn

Estimate the total volume change in the crater that resulted from the summit collapse. Compare your estimate with the [official USGS numbers](https://volcanoes.usgs.gov/vsc/file_mngr/file-192/PrelimSum_LERZ-Summit_2018.pdf). Consider and discuss the sources of uncertainty in your estimate.

Tip: Adding, subtracting, multiplying, and dividing `np.nan` with any number results in a `np.nan` value. To safely work with data containing NaNs, use the `nan` version of numpy functions, like `np.nanmax`, `np.nanmean`, etc. 

## Calculating slope

The slope of the DEM gives the surface inclination (relative to the horizontal plane) at each pixel. Assuming that the surface is a 2D functions of position $s(x, y)$, the slope can be through the gradient of the surface. Mathematically, the slope is the arc-tangent of the norm of the gradient vector:

$$
slope(x, y) = 
\arctan
\left(  
    \sqrt{
        \left(\frac{\partial s}{\partial x}\right)^2 
        + \left(\frac{\partial s}{\partial y}\right)^2
        }
\right)
$$

What we need to do is calculate the x and y derivatives of the surface described by our DEM. When working digitally, we often rely on approximations (numerical methods) to compute these quantities since we don't know the underlying function $s(x, y)$ but only its value in certain locations (the DEM).

A derivative can be approximated using a method called *finite differences*. In this case, we'll use a first-order central difference scheme to approximate our derivatives:

$$
\dfrac{\partial s}{\partial x}(x, y) \approx
\dfrac{s(x + \Delta x, y) - s(x - \Delta x, y)}{2 \Delta x}
$$

This approximation gives us the x derivative at a point (x, y) based on the values in front and behind it. Notice that we can't calculate the derivatives at the borders of our raster DEM since there wouldn't be points in front/behind them to use in the finite differences. For these values, we have to assume that the derivatives take on certain values (boundary conditions). Here, we can assume that they are zero, meaning that the terrain becomes flat at the edges.

For our DEM raster, we can take advantage of clever indexing of numpy arrays to calculate the derivatives. 

### Your turn

Create a function that calculates the slope of a given DEM (include a docstring). Apply your function to calculate the slope of the 2018 DEM. Make a pseudo-color plot of the slope to compare with the 2009 slope.

## Hillshading

This is the processes of adding artificial light and shade to a raster dataset (a grid). It can help visualize suttle changes in data values and produce more realistic looking topographic pseudo-color maps. 

**WARNING:** Hillshading can also cause optical ilusions and over-emphasize features in the data. Watch this [Minute Physics video explaning the optical illusion](https://www.youtube.com/watch?v=V7C318DGB38) to find out more.

Matplotlib has capabilities for doing this with the `matplotlib.colors.LightSource` class. See [this example in the matplotlib gallery](https://matplotlib.org/stable/gallery/specialty_plots/topographic_hillshading.html#sphx-glr-gallery-specialty-plots-topographic-hillshading-py) for more options.